
# --- [1. 브라우저 설정] ---

In [6]:
# 1. 시스템 업데이트 및 구글 크롬 설치
!apt-get update
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install -y ./google-chrome-stable_current_amd64.deb

# 2. 셀레니움 및 필수 라이브러리 설치
!pip install selenium selenium-stealth

Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:5 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,213 B]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 6,955 B in 2s (4,093 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry mi

In [7]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')
options.binary_location = '/usr/bin/google-chrome'

driver = webdriver.Chrome(options=options)

# --- [2. @(C_Idx) 목록 추출 ] ---

In [8]:

def fetch_all_c_indices(limit_pages=1):
    c_idx_set = set()
    print("📋 목록 페이지에서 후기 번호(@)를 검색합니다...")
    for p in range(1, limit_pages + 1):
        list_url = f"https://www.jobkorea.co.kr/starter/review/?Page={p}&OrderBy=11"
        driver.get(list_url)
        time.sleep(3)
        all_links = driver.find_elements(By.TAG_NAME, "a")
        for link in all_links:
            try:
                href = link.get_attribute('href')
                if href and 'C_Idx=' in href:
                    match = re.search(r'C_Idx=(\d+)', href)
                    if match: c_idx_set.add(match.group(1))
            except: continue
    return list(c_idx_set)

# --- [3. 메인 크롤링 엔진] ---

In [5]:
import time, re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoAlertPresentException
from google.colab import files

# =========================
# 설정값 (필요 시 조절)
# =========================
CTGR_CODES = [5]          # 수집할 직무 코드들 (예: [1,2,3,4,5,...])
MAX_LIST_PAGES = 100      # 목록 Page 상한 (100개, 이하 페이지는 질문이 1개 이하이기 때문에 제외함)
MAX_VPAGES = 20           # 상세 VPage 상한 (최대 16page)
SLEEP = 1.2               # 로딩 대기

ORDER_BY = 11
FAVOR = 0
G_ID = 0

# =========================
# 브라우저 설정 (Colab)
# =========================
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.binary_location = "/usr/bin/google-chrome"
driver = webdriver.Chrome(options=options)

def accept_alert_if_any(driver):
    try:
        alert = driver.switch_to.alert
        alert.accept()
        return True
    except NoAlertPresentException:
        return False

def clean_text(t: str) -> str:
    if not t:
        return ""
    t = re.sub(r"\s+", " ", t)
    return t.strip()

def clean_question_text(t: str) -> str:
    t = clean_text(t)
    t = t.replace("Q.", "").strip()
    # 아이콘 텍스트가 섞여도 대부분 공백으로만 들어오니 정리만
    t = re.sub(r"\s+", " ", t).strip()
    return t

def list_url(ctgr_code: int, page: int) -> str:
    return (
        f"https://www.jobkorea.co.kr/starter/Review?"
        f"FavorCo_Stat={FAVOR}&G_ID={G_ID}&OrderBy={ORDER_BY}&Page={page}&Ctgr_Code={ctgr_code}"
    )

def detail_url(c_idx: int, ctgr_code: int, page: int, vpage: int) -> str:
    return (
        f"https://www.jobkorea.co.kr/starter/Review/view?"
        f"C_Idx={c_idx}&Ctgr_Code={ctgr_code}&FavorCo_Stat={FAVOR}&G_ID={G_ID}&Page={page}&VPage={vpage}"
    )

def fetch_all_cidx_for_ctgr(ctgr_code: int, max_pages=200, sleep=1.2):
    """
    특정 Ctgr_Code에 대해 목록 Page를 돌며 C_Idx(@) 전부 수집.
    더 이상 새로운 C_Idx가 안 나오면 자동 종료.
    """
    found = set()
    stagnant = 0  # 새로 안 늘어난 페이지 카운트(연속)
    for p in range(1, max_pages + 1):
        url = list_url(ctgr_code, p)
        driver.get(url)
        accept_alert_if_any(driver)
        time.sleep(sleep)

        # 상세 링크에서 C_Idx 파싱
        links = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/starter/Review/view"]')
        before = len(found)

        for a in links:
            h = a.get_attribute("href")
            if not h:
                continue
            m = re.search(r"[?&]C_Idx=(\d+)", h)
            if m:
                found.add(int(m.group(1)))

        after = len(found)
        print(f"[Ctgr {ctgr_code}] List Page {p} -> C_Idx 누적 {after}개")

        if after == before:
            stagnant += 1
        else:
            stagnant = 0

        # 2페이지 연속으로 증가 없으면 끝으로 간주 (안정성)
        if stagnant >= 2:
            break

    return sorted(found)

def scrape_detail_questions(c_idx: int, ctgr_code: int, page=1, max_vpages=50, sleep=1.2):
    """
    상세(view)에서 VPage(#) 돌면서
    기업명/기간/전형/직군/질문(여러 개면 여러 행) 추출.
    """
    rows = []
    meta_cached = None  # 기업명/기간/전형/직군은 보통 VPage마다 동일하므로 캐시

    for v in range(1, max_vpages + 1):
        url = detail_url(c_idx, ctgr_code, page, v)
        driver.get(url)
        accept_alert_if_any(driver)
        time.sleep(sleep)

        # QNA 아이템 없으면 종료
        qna_items = driver.find_elements(By.CSS_SELECTOR, ".reviewQnaWrap ul li, .qusList li")
        if not qna_items:
            break

        # ---- 메타(기업명/기간/전형/직군) ----
        if meta_cached is None:
            # 기업명
            company = ""
            try:
                company = driver.find_element(By.CSS_SELECTOR, 'a[title="기업 홈 이동"]').text.strip()
            except:
                company = ""

            # 기간/전형
            period, career = "", ""
            try:
                items = [x.text.strip() for x in driver.find_elements(By.CSS_SELECTOR, "span.sCateWrap span.item") if x.text.strip()]
                if len(items) >= 1: period = items[0]
                if len(items) >= 2: career = items[1]
            except:
                pass

            # 직군(제목)
            job_role = ""
            try:
                job_role = clean_text(driver.find_element(By.CSS_SELECTOR, "span.tit").text)
            except:
                job_role = ""

            meta_cached = (company, period, career, job_role)

        company, period, career, job_role = meta_cached

        # ---- 질문 추출 (한 질문 = 한 행) ----
        page_cnt = 0
        for item in qna_items:
            try:
                # 질문은 보통 span.tx 안에 있음
                q_el = item.find_elements(By.CSS_SELECTOR, "span.tx, span.q, .q_tit, .tx")
                if not q_el:
                    continue
                q_text = clean_question_text(q_el[0].text)
                if not q_text:
                    continue

                rows.append({
                    "Ctgr_Code": ctgr_code,
                    "C_Idx": c_idx,
                    "ListPage": page,    # 목록 Page
                    "VPage": v,
                    "기업명": company,
                    "기간": period,      # 예: 2022년 하반기
                    "전형": career,      # 예: 신입
                    "직군": job_role,    # 예: 일반면접 - 앱개발자
                    "질문": q_text
                })
                page_cnt += 1
            except:
                continue

        print(f"   - C_Idx {c_idx} | VPage {v}: 질문 {page_cnt}개")

        # 이 VPage에서 질문 0개면 더 가도 의미 없다고 보고 종료
        if page_cnt == 0:
            break

    return rows

# =========================
# 메인 실행
# =========================
master_rows = []
seen = set()  # 중복 방지용 (ctgr, cidx, vpage, question)

try:
    for ctgr in CTGR_CODES:
        print(f"\n========== Ctgr_Code={ctgr} C_Idx 수집 시작 ==========")
        cidxs = fetch_all_cidx_for_ctgr(ctgr, max_pages=MAX_LIST_PAGES, sleep=SLEEP)
        print(f"Ctgr {ctgr}: C_Idx {len(cidxs)}개 확보")

        for i, c_idx in enumerate(cidxs, 1):
            print(f"\n[{i}/{len(cidxs)}] Ctgr {ctgr} - C_Idx {c_idx} 상세 수집 시작")
            rows = scrape_detail_questions(c_idx, ctgr, page=1, max_vpages=MAX_VPAGES, sleep=SLEEP)

            for r in rows:
                key = (r["Ctgr_Code"], r["C_Idx"], r["VPage"], r["질문"])
                if key in seen:
                    continue
                seen.add(key)
                master_rows.append(r)

    print("\n" + "="*60)
    if master_rows:
        df = pd.DataFrame(master_rows)

        csv_f = "jobkorea_final_questions.csv"
        xlsx_f = "jobkorea_final_questions.xlsx"

        df.to_csv(csv_f, index=False, encoding="utf-8-sig")
        df.to_excel(xlsx_f, index=False)

        print(f"완료! 총 {len(df)}행(질문 기준) 수집")
        display(df.head())

        files.download(csv_f)
        files.download(xlsx_f)
    else:
        print("최종 수집 0건: 선택자/차단/로그인/렌더링 이슈 가능")
    print("="*60)

finally:
    driver.quit()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[358/2000] Ctgr 5 - C_Idx 481 상세 수집 시작
   - C_Idx 481 | VPage 1: 질문 3개

[359/2000] Ctgr 5 - C_Idx 482 상세 수집 시작
   - C_Idx 482 | VPage 1: 질문 30개
   - C_Idx 482 | VPage 2: 질문 17개

[360/2000] Ctgr 5 - C_Idx 485 상세 수집 시작
   - C_Idx 485 | VPage 1: 질문 2개

[361/2000] Ctgr 5 - C_Idx 489 상세 수집 시작
   - C_Idx 489 | VPage 1: 질문 16개

[362/2000] Ctgr 5 - C_Idx 490 상세 수집 시작
   - C_Idx 490 | VPage 1: 질문 4개

[363/2000] Ctgr 5 - C_Idx 494 상세 수집 시작
   - C_Idx 494 | VPage 1: 질문 15개

[364/2000] Ctgr 5 - C_Idx 496 상세 수집 시작
   - C_Idx 496 | VPage 1: 질문 12개

[365/2000] Ctgr 5 - C_Idx 498 상세 수집 시작

[366/2000] Ctgr 5 - C_Idx 499 상세 수집 시작
   - C_Idx 499 | VPage 1: 질문 4개

[367/2000] Ctgr 5 - C_Idx 501 상세 수집 시작
   - C_Idx 501 | VPage 1: 질문 12개

[368/2000] Ctgr 5 - C_Idx 504 상세 수집 시작
   - C_Idx 504 | VPage 1: 질문 1개

[369/2000] Ctgr 5 - C_Idx 506 상세 수집 시작
   - C_Idx 506 | VPage 1: 질문 4개

[370/2000] Ctgr 5 - C_Idx 510 상세 수집 시작
   - C_Idx 510 | VPage 1: 질문 30개
   - C_Idx 510 | VPage

,Ctgr_Code,C_Idx,ListPage,VPage,기업명,기간,전형,직군,질문
0,5,1,1,1,삼성전자㈜,2023년 상반기,신입,일반면접 - 경영·비즈니스기획,[직무면접] 환경공학부를 선택한 이유는?
1,5,1,1,1,삼성전자㈜,2023년 상반기,신입,일반면접 - 경영·비즈니스기획,[직무면접] 삼성전자가 본인에게 어떤 이미지인지?
2,5,1,1,1,삼성전자㈜,2023년 상반기,신입,일반면접 - 경영·비즈니스기획,[직무면접] 환경공학부를 하면서 복수전공을 하지 않은 이유는?
3,5,1,1,1,삼성전자㈜,2023년 상반기,신입,일반면접 - 경영·비즈니스기획,대학 시절 했던 여러 프로젝트들 중에서 협업을 하면서 (프리라이더 등) 힘들었던 경...
4,5,1,1,1,삼성전자㈜,2023년 상반기,신입,일반면접 - 경영·비즈니스기획,남들에게 들은 본인의 단점 한 가지를 이야기해주세요


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


========== Ctgr_Code=5 C_Idx 수집 시작 ==========
[Ctgr 5] List Page 1 -> C_Idx 누적 20개
[Ctgr 5] List Page 2 -> C_Idx 누적 40개
[Ctgr 5] List Page 3 -> C_Idx 누적 60개
[Ctgr 5] List Page 4 -> C_Idx 누적 80개
[Ctgr 5] List Page 5 -> C_Idx 누적 100개
[Ctgr 5] List Page 6 -> C_Idx 누적 120개
[Ctgr 5] List Page 7 -> C_Idx 누적 140개
[Ctgr 5] List Page 8 -> C_Idx 누적 160개
[Ctgr 5] List Page 9 -> C_Idx 누적 180개
[Ctgr 5] List Page 10 -> C_Idx 누적 200개
[Ctgr 5] List Page 11 -> C_Idx 누적 220개
[Ctgr 5] List Page 12 -> C_Idx 누적 240개
[Ctgr 5] List Page 13 -> C_Idx 누적 260개
[Ctgr 5] List Page 14 -> C_Idx 누적 280개
[Ctgr 5] List Page 15 -> C_Idx 누적 300개
[Ctgr 5] List Page 16 -> C_Idx 누적 320개
[Ctgr 5] List Page 17 -> C_Idx 누적 340개
[Ctgr 5] List Page 18 -> C_Idx 누적 360개
[Ctgr 5] List Page 19 -> C_Idx 누적 380개
[Ctgr 5] List Page 20 -> C_Idx 누적 400개
[Ctgr 5] List Page 21 -> C_Idx 누적 420개
[Ctgr 5] List Page 22 -> C_Idx 누적 440개
[Ctgr 5] List Page 23 -> C_Idx 누적 460개
[Ctgr 5] List Page 24 -> C_Idx 누적 480개
[Ctgr 5] List Page 25 -> C_Idx